In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from IPython import *
from tqdm import *

In [8]:
impose_img_dim=(100,100,3)
base_img_hw=(64,64)

In [9]:
#q1_im1
#q1_op1
def im(q,ind,ext=".JPG"):
    return "q"+str(q)+"_im"+str(ind)+ext

def op(q,ind,ext=".JPG"):
    
    return "q"+str(q)+"_op"+str(ind)+ext

def getNames(q):
    return [im(q,i) for i in range(1,5)]+[op(q,i) for i in range(1,5)]


def getEmpImg(size=impose_img_dim):
    img = np.ones(size,dtype=np.float32)*255
    return img

def rotate(img,angle,rotation_point="center"):
    if rotation_point=="center":
        h,w=img.shape[:2]
        cx,cy=(h//2,w//2)
        M = cv2.getRotationMatrix2D((cx, cy),angle,1.0)
        rotated = cv2.warpAffine(img,M,(w, h),borderValue=(255,255,255))
        return rotated

def impose(img,background=None):
    if background is None: background=getEmpImg()
    if img.dtype==np.uint8: img=img.astype(np.float32)
    
    y,x=img.shape[:2]
    by,bx=background.shape[:2]
    lcx=int(max(0,(bx-x)/2))
    lcy=int(max(0,(by-y)/2))
    background[lcy:lcy+y,lcx:lcx+x]=img    
    return background

def imshow(img):
    plt.imshow(img.astype(np.uint8))
    plt.show()

def imread(fname,target_shape=None):
    img=cv2.imread(fname)
    if target_shape is not None: return cv2.resize(img,target_shape,interpolation = cv2.INTER_AREA,)
    return img

def resize(img,target_shape):
    return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)

In [5]:
root_dir="/content/drive/MyDrive/BTP/"
base_img_dir=root_dir+"base_images/"
created_img_dir=root_dir+"created_dataset/"
base_imgs=os.listdir(base_img_dir)
base_imgs=[imread(base_img_dir+i,base_img_hw) for i in base_imgs]

In [6]:
def genImages(base_img,angle):
    l=[base_img]
    for i in range(4):
        l.append(rotate(base_img,(i+1)*angle))

    l.append(rotate(impose(l[-1]),np.random.randint(15,90)))
    l.append(rotate(impose(l[-2]),np.random.randint(15,90)))
    l.append(rotate(impose(l[-3]),np.random.randint(15,90)))

    #l.append(rotate(impose(resize(base_img,(120,70))),np.random.randint(0,angle*2)))
    #l.append(rotate(impose(resize(l[-1],(128,68))),angle))
    #l.append(impose(resize(base_img,(120,70))))
    #l.append(impose(resize(l[1],(128,68))))
    #l.append(impose(resize(l[2],(64,128))))
    return l

In [10]:
dataset=[]
for base_img in base_imgs:
    for base_img_angle in range(0,180):
        curr_img=rotate(impose(base_img),base_img_angle)
        for rotation_angle in range(25,90):
            gen_imgs=[resize(i,base_img_hw) for i in genImages(curr_img,rotation_angle)]